In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [40]:
# XOR dataset
X = torch.tensor([[0,0],[0,1],[1,0],[1,1]], dtype=torch.float32)
y = torch.tensor([[0],[1],[1],[0]], dtype=torch.float32)

class XORNetwork:
    def __init__(self, input_dim=2, hidden_dim=2, output_dim=1):
        self.w1 = torch.randn(input_dim, hidden_dim, requires_grad=True, dtype=torch.float32) * 0.1
        self.b1 = torch.zeros(hidden_dim, requires_grad=True, dtype=torch.float32)
        self.w2 = torch.randn(hidden_dim, output_dim, requires_grad=True, dtype=torch.float32) * 0.1
        self.b2 = torch.zeros(output_dim, requires_grad=True, dtype=torch.float32)

    def forward(self, X):
        self.z1 = torch.matmul(X, self.w1) + self.b1
        self.a1 = torch.sigmoid(self.z1)
        self.z2 = torch.matmul(self.a1, self.w2) + self.b2
        return torch.sigmoid(self.z2)

    def loss(self, y_pred, y):
        return torch.mean((y_pred - y) ** 2)

In [41]:
epochs = 10000
lr = 1.0
model = XORNetwork()

for epoch in range(epochs):
    y_pred = model.forward(X)
    loss = model.loss(y_pred, y)
    loss.backward()

    with torch.no_grad():
        model.w1 -= lr * model.w1.grad
        model.b1 -= lr * model.b1.grad
        model.w2 -= lr * model.w2.grad
        model.b2 -= lr * model.b2.grad

    model.w1.grad.zero_()
    model.b1.grad.zero_()
    model.w2.grad.zero_()
    model.b2.grad.zero_()

    if (epoch+1) % 1000 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.6f}')

print("\nPredictions after training:")
print(model.forward(X))

/tmp/ipython-input-41-1598292094.py:11: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /pytorch/build/aten/src/ATen/core/TensorBody.h:489.)
  model.w1 -= lr * model.w1.grad


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df. shape

(569, 33)

In [ ]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [ ]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

array([[-0.90810768,  1.13448886, -0.91876617, ..., -1.1846073 ,
         0.27764974, -0.42149146],
       [-1.45002726, -1.07646735, -1.33040643, ..., -0.95613438,
        -1.01658972,  1.32724805],
       [-1.08201562, -0.42034241, -1.07527098, ..., -1.13852242,
        -0.02943068, -0.32192104],
       ...,
       [-0.78468915,  0.12257787, -0.8205431 , ..., -1.24945639,
        -0.5294412 , -0.42856477],
       [-1.49546773, -0.80154176, -1.45063799, ..., -0.09838516,
         0.06381888,  0.96270061],
       [ 3.27409777, -0.42496301,  3.3732967 , ...,  2.46255318,
         1.29214055,  0.22544422]])

In [ ]:
y_train

,diagnosis
549,B
71,B
217,B
360,B
253,M
...,...
186,M
433,M
522,B
114,B


In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
y_train

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

Converting Numpy Arrays to Pytorch Tensors

In [ ]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [ ]:
X_train_tensor.shape[0]

455

In [ ]:
class Simple_NeuralNetwork():
  def __init__(self,X):
    self.weights=torch.rand(X.shape[1],1,dtype=torch.float64,requires_grad=True)
    self.bias=torch.rand(1,dtype=torch.float64,requires_grad=True)

  def forward(self,X):
    #z=wx+b  equation of linear regression, or for fitting

    z = torch.matmul(X, self.weights) + self.bias
    #Applying Sigmoid Function: In neural networks, the sigmoid function is a
    #mathematical function that squashes values between 0 and 1, making it
    #useful as an activation function, particularly in binary classification.
    #It transforms any real-valued number into a probability between 0 and 1.

    y_pred=torch.sigmoid(z)
    return y_pred

  def loss(self,y_pred,y):
    # Clamp predictions to avoid log(0)
    epsilon = 1e-7
    y_pred=torch.clamp(y_pred,epsilon,1-epsilon)
    #Calculating Loss
    # The BCE loss : (y * log(p) + (1 - y) * log(1 - p)) evalues this for all samples then does a mean
    #Binary Cross Entropy (BCE) loss is a loss function used in
    #binary classification tasks to quantify the difference between
    #predicted probabilities and actual binary labels. It's also known as log loss.
    loss=-(y*torch.log(y_pred)+(1-y)*torch.log(1-y_pred)).mean()
    return loss

  def acuuracy_score():
    temp=0
    for i in range(len(y_pred)):
      if(y_pred[i]>0.5):
        y_pred[i]=1
      else:
        y_pred[i]=0

      if(y_pred[i]==y_test_tensor[i]):
        temp=temp+1





Parameter Settings

In [ ]:
lr=0.001
epochs=1000
# create model
model = Simple_NeuralNetwork(X_train_tensor)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model.forward(X_train_tensor)

  # loss calculate
  loss = model.loss(y_pred, y_train_tensor)

  # backward pass
  loss.backward()

  # parameters update
  #model.weights -= learning_rate * model.weights.grad
  #✔ In-place update — modifies weights directly, but can interfere with autograd if misused.

  #model.weights = model.weights - (learning_rate * model.weights.grad)
  #✘ Out-of-place — reassigns a new tensor, which may break autograd tracking unless properly wrapped as a torch.nn.Parameter.

  with torch.no_grad():
    model.weights -= lr * model.weights.grad
    model.bias -= lr * model.bias.grad

  # zero gradients
  model.weights.grad.zero_()
  model.bias.grad.zero_()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 3.4112290393703484
Epoch: 2, Loss: 3.410112650885825
Epoch: 3, Loss: 3.408996406355902
Epoch: 4, Loss: 3.4078803059258544
Epoch: 5, Loss: 3.406764349748392
Epoch: 6, Loss: 3.405648537966154
Epoch: 7, Loss: 3.4045328707302227
Epoch: 8, Loss: 3.4034173481858327
Epoch: 9, Loss: 3.402301970485193
Epoch: 10, Loss: 3.401186737775178
Epoch: 11, Loss: 3.40007165020341
Epoch: 12, Loss: 3.3989567079148184
Epoch: 13, Loss: 3.3978419110643845
Epoch: 14, Loss: 3.3967272598001443
Epoch: 15, Loss: 3.395608062034751
Epoch: 16, Loss: 3.394471576522996
Epoch: 17, Loss: 3.3933210448594187
Epoch: 18, Loss: 3.392170664096201
Epoch: 19, Loss: 3.3910204343943615
Epoch: 20, Loss: 3.389870355914104
Epoch: 21, Loss: 3.3887204288159167
Epoch: 22, Loss: 3.38757065325389
Epoch: 23, Loss: 3.386421029383511
Epoch: 24, Loss: 3.3852715573723255
Epoch: 25, Loss: 3.3841222373657573
Epoch: 26, Loss: 3.3829730695415683
Epoch: 27, Loss: 3.381824054043012
Epoch: 28, Loss: 3.380675191036535
Epoch: 29, Loss: 3

In [ ]:
#Checking the performance of the trained model
temp=0
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)

for i in range(len(y_pred)):
  if(y_pred[i]>0.5):
    y_pred[i]=1
  else:
    y_pred[i]=0

  if(y_pred[i]==y_test_tensor[i]):
    temp=temp+1


print(f' Accuracy is {(temp/y_test_tensor.shape[0])*100}')

 Accuracy is 80.7017543859649


Torch with NN Module - Constructing the Neural Network Module using NN.

A custom module in PyTorch is a user-defined module that is built using the PyTorch library's built-in neural network module, torch.nn.Module. It's a way of creating new modules by combining and extending the functionality provided by existing PyTorch modules.

The torch.nn.Module class provides a convenient way to create custom modules because it includes some key features that are important for building neural networks, such as the ability to keep track of learnable parameters and the ability to perform automatic differentiation (for computing gradients during training).

By creating a new class that inherits from torch.nn.Module, and defining an __init__ method to initialize the module's parameters, and forward method that perform the computation, we can create our own custom module. These custom modules can be used just like any of the built-in PyTorch modules, such as torch.nn.Module or torch.nn.Conv2d, and can be included in a larger model architecture.

LINK: https://chatgpt.com/share/68493a42-2d18-800e-99f9-19d07332fb4d

In [ ]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))



# create model class
import torch
import torch.nn as nn

class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

In [ ]:
learning_rate = 0.001
epochs = 1000

In [ ]:
# define loss function
loss_function = nn.BCELoss()

In [ ]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loop
for epoch in range(epochs):

  # forward pass
  y_pred = model(X_train_tensor) #when you do this the forward pass is automatically triggered. this is a magic function.
  #Link: https://www.geeksforgeeks.org/dunder-magic-methods-python/

  # loss calculate
  loss = loss_function(y_pred, y_train_tensor.view(-1,1))

  #y_train_tensor.view(-1, 1) is a PyTorch operation that reshapes the tensor y_train_tensor into a 2-dimensional tensor with 1 column, and the number of rows is inferred automatically.
  #-1: PyTorch will infer the correct size for this dimension based on the total number of elements and the other dimensions specified. It's a placeholder for "whatever size fits". 1: This is the number of columns — you're explicitly saying that you want each target value to be a separate row, and each row should have one value.

  # clear gradients
  optimizer.zero_grad()

  # backward pass
  loss.backward()

  # parameters update
  optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.6175392270088196
Epoch: 2, Loss: 0.6160126328468323
Epoch: 3, Loss: 0.6144943833351135
Epoch: 4, Loss: 0.6129844188690186
Epoch: 5, Loss: 0.6114826202392578
Epoch: 6, Loss: 0.6099890470504761
Epoch: 7, Loss: 0.6085036993026733
Epoch: 8, Loss: 0.6070261597633362
Epoch: 9, Loss: 0.6055569052696228
Epoch: 10, Loss: 0.6040953993797302
Epoch: 11, Loss: 0.6026418805122375
Epoch: 12, Loss: 0.6011962890625
Epoch: 13, Loss: 0.5997583866119385
Epoch: 14, Loss: 0.5983282327651978
Epoch: 15, Loss: 0.5969058275222778
Epoch: 16, Loss: 0.5954910516738892
Epoch: 17, Loss: 0.5940837860107422
Epoch: 18, Loss: 0.5926841497421265
Epoch: 19, Loss: 0.5912920236587524
Epoch: 20, Loss: 0.5899072885513306
Epoch: 21, Loss: 0.5885298848152161
Epoch: 22, Loss: 0.5871598720550537
Epoch: 23, Loss: 0.5857971906661987
Epoch: 24, Loss: 0.5844416618347168
Epoch: 25, Loss: 0.5830933451652527
Epoch: 26, Loss: 0.5817521810531616
Epoch: 27, Loss: 0.5804181098937988
Epoch: 28, Loss: 0.5790910720825195
Epoc

In [ ]:
# model evaluation
with torch.no_grad():
  y_pred = model.forward(X_test_tensor)
  y_pred = (y_pred > 0.5).float()
  accuracy = (y_pred == y_test_tensor).float().mean()
  print(f'Accuracy: {accuracy.item()}')

Accuracy: 0.5120037198066711


In [ ]:
#showcasing the weights.
model.linear.weight.shape

torch.Size([1, 30])

In [ ]:
model.linear.bias.shape

torch.Size([1])

In [ ]:
!pip install torchinfo
from torchinfo import summary

In [ ]:
summary(model,input_size=(455,30))

Layer (type:depth-idx)                   Output Shape              Param #
MySimpleNN                               [455, 1]                  --
├─Linear: 1-1                            [455, 1]                  31
├─Sigmoid: 1-2                           [455, 1]                  --
Total params: 31
Trainable params: 31
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.05
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.06

===Constructing a Neural Network with a Hidden Layer===<br>
Layer 1 (30x5)-->ReLU; Layer 2(5x1)-->Sigmoid; Output Layer (1)


In [ ]:
# create model class
import torch
import torch.nn as nn

# nn.Module, which is the base class for all neural networks in PyTorch.
#a. The parent class (nn.Module) does critical setup, including:

#b. Registering layers (like nn.Linear, nn.Conv2d, etc.).

#c. Making .parameters() work, so the optimizer knows what to update.

#d. Enabling features like .cuda(), .eval(), .state_dict(), etc.

class TwoLayerNN(nn.Module):
  def __init__(self,num_feature):
    #super() is a built-in function in Python that returns a temporary object of the parent class,
    #so you can call its methods (like __init__) without directly naming the parent class.
    #here when we are writing super().__init__() it is actually triggering super().__init__(self)
    super().__init__()
    #Creating one input layer and a hidden. Linear layer means a fully connected layer.
    self.input_layer=nn.Linear(num_feature,5)
    self.hidden_layer=nn.Linear(5,1)
    self.relu=nn.ReLU()
    self.sigmoid=nn.Sigmoid()

  def forward(self,input_features):
    out=self.input_layer(input_features)
    out=self.relu(out)
    out=self.hidden_layer(out)
    out=self.sigmoid(out)
    return out



In [ ]:
lr=0.001
epochs=300
loss_function=nn.BCELoss()

In [ ]:
model = TwoLayerNN(X_train_tensor.shape[1])

In [ ]:
#for param in model.parameters():
    #if param.grad is None:
        #continue
    #param.data = param.data - lr * param.grad.data

#===Updating parameters using stochastic gradient descent=====

optimizer=torch.optim.SGD(model.parameters(),lr=lr)

for epoch in range(epochs):
  y_pred=model(X_train_tensor)
  loss=loss_function(y_pred,y_train_tensor.view(-1,1))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  print(f'Epoch is {epoch+1} and loss is {loss}')




Epoch is 1 and loss is 0.7289665341377258
Epoch is 2 and loss is 0.7288199663162231
Epoch is 3 and loss is 0.72867351770401
Epoch is 4 and loss is 0.7285270094871521
Epoch is 5 and loss is 0.7283806800842285
Epoch is 6 and loss is 0.7282345294952393
Epoch is 7 and loss is 0.7280884385108948
Epoch is 8 and loss is 0.7279424071311951
Epoch is 9 and loss is 0.7277966737747192
Epoch is 10 and loss is 0.7276511192321777
Epoch is 11 and loss is 0.7275057435035706
Epoch is 12 and loss is 0.7273603677749634
Epoch is 13 and loss is 0.7272149324417114
Epoch is 14 and loss is 0.7270695567131042
Epoch is 15 and loss is 0.7269248366355896
Epoch is 16 and loss is 0.726780354976654
Epoch is 17 and loss is 0.7266359329223633
Epoch is 18 and loss is 0.7264918088912964
Epoch is 19 and loss is 0.7263476252555847
Epoch is 20 and loss is 0.7262036204338074
Epoch is 21 and loss is 0.72605961561203
Epoch is 22 and loss is 0.7259157299995422
Epoch is 23 and loss is 0.7257719039916992
Epoch is 24 and loss is 0

In [ ]:
with torch.no_grad():
  y_pred=model.forward(X_test_tensor)
  y_pred=(y_pred>0.5).float()
  accuracy=(y_pred==y_test_tensor).float().mean()
  print(f'Accuracy is {accuracy}')

Accuracy is 0.4566020369529724


In [ ]:
summary(model,input_size=(455,30))

Layer (type:depth-idx)                   Output Shape              Param #
TwoLayerNN                               [455, 1]                  --
├─Linear: 1-1                            [455, 5]                  155
├─ReLU: 1-2                              [455, 5]                  --
├─Linear: 1-3                            [455, 1]                  6
├─Sigmoid: 1-4                           [455, 1]                  --
Total params: 161
Trainable params: 161
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.07
Input size (MB): 0.05
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.08

To Simplify the work we will put the (self.input_layer=nn.Linear(num_feature,5)
    self.hidden_layer=nn.Linear(5,1)
    self.relu=nn.ReLU()
    self.sigmoid=nn.Sigmoid()) into a sequential container.

In PyTorch, containers are classes or data structures designed to hold and organize neural network components such as layers, modules, parameters, or other sub-networks. They help in structuring complex neural network architectures and managing components efficiently.
Link:https://medium.com/@karuneshu21/containers-pytorch-10bc14a63aa4

What is nn.Sequential?<br>
Ans: nn.Sequential is a container module in PyTorch that lets you stack layers and operations in the exact order you want them to run, without needing to manually define the forward() method.

In [ ]:
# create model class
import torch
import torch.nn as nn



class TwoLayerNN(nn.Module):
  def __init__(self,num_feature):

    super().__init__()
    #Making Sequential Container

    self.network=nn.Sequential(
        nn.Linear(num_feature,5),
        nn.ReLU(),
        nn.Linear(5,1),
        nn.Sigmoid()
    )


  def forward(self,input_features):
    out=self.network(input_features)
    return out



In [ ]:
lr=0.001
epochs=300
loss_function=nn.BCELoss()
model = TwoLayerNN(X_train_tensor.shape[1])


optimizer=torch.optim.SGD(model.parameters(),lr=lr)

for epoch in range(epochs):
  y_pred=model(X_train_tensor)
  loss=loss_function(y_pred,y_train_tensor.view(-1,1))
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  print(f'Epoch is {epoch+1} and loss is {loss}')




Epoch is 1 and loss is 0.6784456968307495
Epoch is 2 and loss is 0.6781927943229675
Epoch is 3 and loss is 0.677940309047699
Epoch is 4 and loss is 0.6776874661445618
Epoch is 5 and loss is 0.6774352192878723
Epoch is 6 and loss is 0.6771834492683411
Epoch is 7 and loss is 0.6769318580627441
Epoch is 8 and loss is 0.6766799688339233
Epoch is 9 and loss is 0.6764283180236816
Epoch is 10 and loss is 0.6761767268180847
Epoch is 11 and loss is 0.6759248971939087
Epoch is 12 and loss is 0.6756730675697327
Epoch is 13 and loss is 0.6754213571548462
Epoch is 14 and loss is 0.6751700639724731
Epoch is 15 and loss is 0.6749191284179688
Epoch is 16 and loss is 0.6746683120727539
Epoch is 17 and loss is 0.6744175553321838
Epoch is 18 and loss is 0.6741669774055481
Epoch is 19 and loss is 0.6739153265953064
Epoch is 20 and loss is 0.6736639142036438
Epoch is 21 and loss is 0.6734126210212708
Epoch is 22 and loss is 0.6731615662574768
Epoch is 23 and loss is 0.6729108691215515
Epoch is 24 and loss 

In [ ]:
with torch.no_grad():
  y_pred=model.forward(X_test_tensor)
  y_pred=(y_pred>0.5).float()
  accuracy=(y_pred==y_test_tensor).float().mean()
  print(f'Accuracy is {accuracy}')

Accuracy is 0.5517082214355469
